# Fabric RTI MCP Client
This notebook demonstrates how to connect to and use the Microsoft Fabric RTI MCP server from VS Code.

In [ ]:
# Install required packages
!pip install mcp httpx asyncio

In [ ]:
import asyncio
import json
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

async def connect_to_mcp_server():
    """Connect to the Fabric RTI MCP server"""
    server_params = StdioServerParameters(
        command="uvx",
        args=["microsoft-fabric-rti-mcp"]
    )
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()
            
            # List available tools
            tools = await session.list_tools()
            print("Available MCP tools:")
            for tool in tools.tools:
                print(f"  - {tool.name}: {tool.description}")
            
            return session

# Run the connection test
await connect_to_mcp_server()

In [ ]:
# Example: List Eventstreams
async def list_eventstreams(workspace_id: str, auth_token: str):
    server_params = StdioServerParameters(
        command="uvx",
        args=["microsoft-fabric-rti-mcp"]
    )
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            result = await session.call_tool(
                "eventstream_list",
                {
                    "workspace_id": workspace_id,
                    "authorization_token": auth_token
                }
            )
            
            return result.content

# Usage (replace with your actual credentials):
# result = await list_eventstreams("your-workspace-id", "Bearer your-token")
# print(json.dumps(result, indent=2))

In [ ]:
# Example: Query Kusto
async def query_kusto(cluster_uri: str, query: str, database: str = None):
    server_params = StdioServerParameters(
        command="uvx",
        args=["microsoft-fabric-rti-mcp"]
    )
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            args = {
                "cluster_uri": cluster_uri,
                "query": query
            }
            if database:
                args["database"] = database
                
            result = await session.call_tool("kusto_query", args)
            return result.content

# Usage (replace with your actual cluster and query):
# result = await query_kusto(
#     "https://your-cluster.kusto.windows.net", 
#     "MyTable | take 10"
# )
# print(json.dumps(result, indent=2))